In [11]:
import requests
import pandas as pd
import pickle
import json

import sys
sys.path.append('..\\src')
import citybikes_parsing


# CityBikes

### Query the City Bikes API for bike stations in San Francisco

In [4]:
url = 'https://api.citybik.es/v2/networks/bay-wheels'
response = requests.get(url)
print('status:', response.status_code)
res = response.json()
print(res)

status: 200
{'network': {'company': ['Motivate LLC'], 'gbfs_href': 'https://gbfs.baywheels.com/gbfs/gbfs.json', 'href': '/v2/networks/bay-wheels', 'id': 'bay-wheels', 'location': {'city': 'San Francisco Bay Area, CA', 'country': 'US', 'latitude': 37.7141454, 'longitude': -122.25}, 'name': 'Bay Wheels', 'stations': [{'empty_slots': 6, 'extra': {'ebikes': 0, 'has_ebikes': True, 'last_updated': 1712082426, 'payment': ['key', 'creditcard'], 'payment-terminal': True, 'rental_uris': {'android': 'https://sfo.lft.to/lastmile_qr_scan', 'ios': 'https://sfo.lft.to/lastmile_qr_scan'}, 'renting': 1, 'returning': 1, 'slots': 15, 'uid': 'fd89514c-f878-4cd5-8113-8e5beead44de'}, 'free_bikes': 9, 'id': 'd0e8f4f1834b7b33a3faf8882f567ab8', 'latitude': 37.849735, 'longitude': -122.270582, 'name': 'Harmon St at Adeline St', 'timestamp': '2024-04-02T18:29:27.066000Z'}, {'empty_slots': 3, 'extra': {'ebikes': 6, 'has_ebikes': True, 'last_updated': 1712082446, 'payment': ['key', 'creditcard'], 'payment-terminal

In [11]:
with open('../data/citybikes_sanfran.pickle', 'wb') as citybikes_sanfran:
    pickle.dump(res, citybikes_sanfran)

In [12]:
# UNPICKLE CITYBIKES API RESPONSE JSON 
with open('../data/citybikes_sanfran.pickle', 'rb') as citybikes_sanfran:
    citybikes_sanfran2 = pickle.load(citybikes_sanfran)

print(citybikes_sanfran2)

{'network': {'company': ['Motivate LLC'], 'gbfs_href': 'https://gbfs.baywheels.com/gbfs/gbfs.json', 'href': '/v2/networks/bay-wheels', 'id': 'bay-wheels', 'location': {'city': 'San Francisco Bay Area, CA', 'country': 'US', 'latitude': 37.7141454, 'longitude': -122.25}, 'name': 'Bay Wheels', 'stations': [{'empty_slots': 6, 'extra': {'ebikes': 0, 'has_ebikes': True, 'last_updated': 1712082426, 'payment': ['key', 'creditcard'], 'payment-terminal': True, 'rental_uris': {'android': 'https://sfo.lft.to/lastmile_qr_scan', 'ios': 'https://sfo.lft.to/lastmile_qr_scan'}, 'renting': 1, 'returning': 1, 'slots': 15, 'uid': 'fd89514c-f878-4cd5-8113-8e5beead44de'}, 'free_bikes': 9, 'id': 'd0e8f4f1834b7b33a3faf8882f567ab8', 'latitude': 37.849735, 'longitude': -122.270582, 'name': 'Harmon St at Adeline St', 'timestamp': '2024-04-02T18:29:27.066000Z'}, {'empty_slots': 3, 'extra': {'ebikes': 6, 'has_ebikes': True, 'last_updated': 1712082446, 'payment': ['key', 'creditcard'], 'payment-terminal': True, 're

## Figure out how to extract the info that I need from the JSON

In [40]:
# the last index [x] refers to individal stations in the Bay Wheels network
citybikes_sanfran2['network']['stations'][0]

{'empty_slots': 6,
 'extra': {'ebikes': 0,
  'has_ebikes': True,
  'last_updated': 1712082426,
  'payment': ['key', 'creditcard'],
  'payment-terminal': True,
  'rental_uris': {'android': 'https://sfo.lft.to/lastmile_qr_scan',
   'ios': 'https://sfo.lft.to/lastmile_qr_scan'},
  'renting': 1,
  'returning': 1,
  'slots': 15,
  'uid': 'fd89514c-f878-4cd5-8113-8e5beead44de'},
 'free_bikes': 9,
 'id': 'd0e8f4f1834b7b33a3faf8882f567ab8',
 'latitude': 37.849735,
 'longitude': -122.270582,
 'name': 'Harmon St at Adeline St',
 'timestamp': '2024-04-02T18:29:27.066000Z'}

In [53]:
# there are 555 bike stations in the network 
len(citybikes_sanfran2['network']['stations'])

555

In [54]:
# name of bike stn
citybikes_sanfran2['network']['stations'][0]['name']

# id of bike stn
citybikes_sanfran2['network']['stations'][0]['id']

# timestamp for when data was returned by the API
citybikes_sanfran2['network']['stations'][0]['timestamp']

# longitude and latitude
citybikes_sanfran2['network']['stations'][0]['longitude']
citybikes_sanfran2['network']['stations'][0]['latitude']

# total bike slots
citybikes_sanfran2['network']['stations'][0]['extra']['slots']

# free bikes
citybikes_sanfran2['network']['stations'][0]['free_bikes']

# empty bike slots
citybikes_sanfran2['network']['stations'][0]['empty_slots']

# has ebikes
citybikes_sanfran2['network']['stations'][0]['extra']['has_ebikes']

# payment types
citybikes_sanfran2['network']['stations'][0]['extra']['payment']

# renting and returning
citybikes_sanfran2['network']['stations'][0]['extra']['renting']
citybikes_sanfran2['network']['stations'][0]['extra']['returning']

# ebikes
citybikes_sanfran2['network']['stations'][0]['extra']['ebikes']

0

In [13]:
cols = ["name", "id", "timestamp", "longitude", "latitude", "slots", "free_bikes",
        "empty_slots", "has_ebikes", "ebikes", "payment", "renting", "returning"]

citybikes_df = pd.DataFrame(columns=cols)

print(citybikes_df)

Empty DataFrame
Columns: [name, id, timestamp, longitude, latitude, slots, free_bikes, empty_slots, has_ebikes, ebikes, payment, renting, returning]
Index: []


## Parse the JSON & insert relevant info into a new df

In [14]:
# Parse dict returned by Citybikes API into a blank df with appropriate headers, using a function from another module 
citybikes_filled_df = citybikes_parsing.citybikes_parsing(citybikes_sanfran2, citybikes_df)
citybikes_filled_df.head()

,name,id,timestamp,longitude,latitude,slots,free_bikes,empty_slots,has_ebikes,ebikes,payment,renting,returning
0,Harmon St at Adeline St,d0e8f4f1834b7b33a3faf8882f567ab8,2024-04-02T18:29:27.066000Z,-122.270582,37.849735,15,9,6,True,0,"[key, creditcard]",1,1
1,Fountain Alley at S 2nd St,983514094dd808b1604da2dcfc2d09af,2024-04-02T18:29:26.582000Z,-121.889277,37.336188,11,8,3,True,6,"[key, creditcard]",1,1
2,Oak St at 1st St,da17603652106fda93da4e255a5b0a22,2024-04-02T18:29:27.044000Z,-121.88109,37.322125,23,14,9,True,7,"[key, creditcard]",1,1
3,Bestor Art Park,7a21c92b3b4cd2f7759107b4fdebf869,2024-04-02T18:29:26.797000Z,-121.874119,37.323678,15,12,3,True,3,"[key, creditcard]",1,1
4,5th St at Virginia St,ce34d38fb230a23c1ced12d1e16df294,2024-04-02T18:29:26.971000Z,-121.87712,37.325998,27,19,7,True,12,"[key, creditcard]",1,1


In [18]:
citybikes_filled_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 555 entries, 0 to 554
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         555 non-null    object
 1   id           555 non-null    object
 2   timestamp    555 non-null    object
 3   longitude    555 non-null    object
 4   latitude     555 non-null    object
 5   slots        555 non-null    object
 6   free_bikes   555 non-null    object
 7   empty_slots  555 non-null    object
 8   has_ebikes   555 non-null    object
 9   ebikes       555 non-null    object
 10  payment      533 non-null    object
 11  renting      555 non-null    object
 12  returning    555 non-null    object
dtypes: object(13)
memory usage: 60.7+ KB


In [15]:
with open('../data/citybikes_filled_df.pickle', 'wb') as df:
    pickle.dump(citybikes_filled_df, df)

In [1]:
# UNPICKLE CITYBIKES DF 
with open('../data/citybikes_filled_df.pickle', 'rb') as df:
    citybikes_df = pickle.load(df)

NameError: name 'pickle' is not defined